# Expliquer le vote populiste en France

## Première partie : Le vote comme expression d'opinions subjectives

« L'isoloir planté dans une salle d'école ou de mairie est le symbole de toutes les trahisons que l'individu peut commettre envers les groupes dont il fait partie. ». Cette phrase suffirait presque à résumer l'argumentaire de Sartre dans son célèbre texte "Élections, piège à cons", publié dans la revue Les Temps Modernes en 1973. Pour Sartre, le fait que la pratique du vote soit dans nos démocraties modernes un acte individuel empêche les classes sociales de porter collectivement leur intérêt, car chacun peut se défiler au moment de prendre la décision importante qui est celle d'élire les réprésentants du peuple. En effet, le ressenti individuel du votant peut ne pas correspondre à l'analyse objective que ferait un observateur extérieur de ce que devrait chosir le votant en fonction de ses caractéristiques socio-économiques objectives (niveau d'éducation, revenu,... en bref sa classe sociale). Or, c'est bien ce ressenti individuel et subjectif qui détermine en dernière instance le vote. Les caractéristiques individuelles objectives ne sont que des facteurs explicatifs parmi d'autres de ce ressenti. C'est pourquoi la sociologie électorale ne se contente pas de lier l'environnement social des individus à leur vote mais cherche également à collecter leur ressenti subjectif sur le monde dans lequel ils vivent.

La base de données à notre disposition permet justement de prendre en compte ce ressenti subjectif. En effet, la plupart des variables sont des réponses à des questions d'opinion. Le but de cette partie est de lier ce ressenti subjectif aux déclarations de vote (en particulier du vote FN) des individus. Devant l'étendue des variables et thématiques disponibles, nous avons construit 5 variables subjectives synthétiques nous semblant pertinentes pour expliquer le vote FN.

Avant d'expliquer comment nous construisons ces variables, on peut dans un premier temps décrire brièvement la base en question à l'aide de quelques statistiques descriptives.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels as sm
import statsmodels.api as st
import statsmodels.formula.api as smf
import urllib
import bs4

In [8]:
base_indiv = pd.read_spss('base_indiv.sav', convert_categoricals=False)
base_indiv.shape

(2070, 539)

Comme on peut le voir, la base est extrêmement large. À l'aide du descriptif de chaque variable, que l'on peut trouver en ligne (http://nesstar.ess.nsd.uib.no/webview/index.jsp?v=2&submode=abstract&study=http%3A%2F%2F129.177.90.83%3A80%2Fobj%2FfStudy%2FESS9e02.0&mode=documentation&top=yes), nous avons présélectionné un sous-ensemble de 37 variables, qui recouvrent à notre sens l'essentiel des variables subjectives et objectives pertinentes pour notre problématique. Nous avons également veillé à ne pas chosir des variables redondantes. La description de chaque variable est disponible dans un document annexe.

In [9]:
base_indiv = base_indiv[["ess8_id", "nuts2", "ppltrst", "cptppola", "trstplt", "ccnthum", "polintr", "vote", "pbldmn","bctprd","pstplonl", "mbtru", "stflife", "stfedu", "stfhlth", "aesfdrk", "health",  "gincdif", "euftf", "imwbcnt", "ipstrgv", "mnrgtjb", "freehms", "imptrad", "rlgblg", "blgetmg", "crmvct", "prtvtcfr", "gndr", "agea", "chldhm", "edlvdfr","uemp3m", "tporgwk", "hinctnta", "lrscale", "pspwght"]]
base_indiv = base_indiv.rename(columns={'pspwght' : 'weight'})
base_indiv.describe()

,ppltrst,cptppola,trstplt,ccnthum,polintr,vote,pbldmn,bctprd,pstplonl,mbtru,...,prtvtcfr,gndr,agea,chldhm,edlvdfr,uemp3m,tporgwk,hinctnta,lrscale,weight
count,2070.000000,2055.000000,2059.000000,2024.000000,2069.000000,2003.00000,2068.000000,2065.000000,2068.000000,2063.000000,...,1059.000000,2070.000000,2069.000000,2070.000000,2068.000000,2066.000000,1936.000000,1884.000000,1916.000000,2070.000000
mean,4.583575,2.195134,2.893152,3.614625,2.559691,1.52671,1.856867,1.676513,1.803191,2.713524,...,8.600567,1.539614,52.377960,1.701449,10.130561,1.610842,3.620351,4.855626,5.079854,1.000000
std,2.143143,0.943060,2.094522,2.907148,0.970024,0.70722,0.350294,0.467920,0.397683,0.591605,...,3.167957,0.498549,18.929428,0.457733,6.134469,0.487677,1.176417,2.740380,2.409379,0.691451
min,0.000000,1.000000,0.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.228504
25%,3.000000,1.000000,1.000000,3.000000,2.000000,1.00000,2.000000,1.000000,2.000000,3.000000,...,9.000000,1.000000,38.000000,1.000000,6.000000,1.000000,3.000000,2.000000,3.000000,0.539700
50%,5.000000,2.000000,3.000000,3.000000,3.000000,1.00000,2.000000,2.000000,2.000000,3.000000,...,9.000000,2.000000,53.000000,2.000000,7.000000,2.000000,4.000000,5.000000,5.000000,0.803473
75%,6.000000,3.000000,4.000000,4.000000,3.000000,2.00000,2.000000,2.000000,2.000000,3.000000,...,10.000000,2.000000,67.000000,2.000000,15.000000,2.000000,4.000000,7.000000,7.000000,1.203349
max,10.000000,5.000000,10.000000,55.000000,4.000000,3.00000,2.000000,2.000000,2.000000,3.000000,...,16.000000,2.000000,99.000000,2.000000,26.000000,2.000000,6.000000,10.000000,10.000000,4.000000


On peut faire apparaître les valeurs manquantes de façon plus lisible.

In [10]:
base_indiv.isnull().sum()

ess8_id        0
nuts2          0
ppltrst        0
cptppola      15
trstplt       11
ccnthum       46
polintr        1
vote          67
pbldmn         2
bctprd         5
pstplonl       2
mbtru          7
stflife        2
stfedu        20
stfhlth        5
aesfdrk       48
health         2
gincdif       14
euftf         81
imwbcnt       27
ipstrgv       21
mnrgtjb        6
freehms       19
imptrad       14
rlgblg         8
blgetmg       39
crmvct         2
prtvtcfr    1011
gndr           0
agea           1
chldhm         0
edlvdfr        2
uemp3m         4
tporgwk      134
hinctnta     186
lrscale      154
weight         0
dtype: int64

On s'aperçoit que seule la variable prtvtcfr (parti choisi aux dernières élections nationales) a plus de 10% de valeurs manquantes. Comment expliquer cette lacune pour cette variable ? La variable vote (qui prend la valeur 1 quand l'individu a voté aux dernières élections, 2 quand il n'a pas voté et 3 quand il ne pouvait pas voter) donne un premier élément de réponse à cette question.

In [14]:
base_indiv['vote'].value_counts()

1.0    1199
2.0     553
3.0     251
Name: vote, dtype: int64

In [21]:
base_indiv['vote'][base_indiv['vote']==1].count() - base_indiv[base_indiv['prtvtcfr'].notnull()]['prtvtcfr'].count()

140

In [19]:
base_indiv['prtvtcfr'].notnull().count()

2070

Les valeurs manquantes s'expliquent donc par les abstentionnistes (553), les non-inscrits sur les listes électorales (251) et ceux n'ayant pas voulu dire pour qui ils avaient voté (140). 

### Proxy confiance

### Modèles variables subjectives